### Create document loader

Install specific dependencies and packages for document handling

In [ ]:
%pip install --q unstructured langchain
%pip install --q "unstructured[all-docs]"
%pip install --q langchain-community

In [1]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader
import os
from dotenv import load_dotenv
load_dotenv()

True

### Load documents

In [2]:
local_path = os.getenv("LOCAL_PATH")

# Local PDF file uploads
if local_path:
  loader = UnstructuredPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print("Upload a PDF file")

let's explore docs

In [20]:
# data type must be document
type(data[0])

langchain_core.documents.base.Document

In [3]:
# Preview first page
data[0].page_content

'https://doi.org/10.23850/2422068X.1182\n\nUn análisis de la estructura productiva del Alto Magdalena\n\nAn analysis of the productive structure of Alto Magdalena\n\ndIana PatrIcIa GaLIndo áVILa1 GLorIa ISaBeL PrecIado trUJILLo2\n\nRESUMEN Este artículo da a conocer el desarrollo del proyecto de investigación que permitió la caracterización empresarial de la provincia del Alto Magdalena, cuyo objetivo es determinar los elementos esenciales que influyen en el crecimiento económico de la región.\n\nDentro del desarrollo de la investigación se usó como referente metodológico el libro Metodología de la investigación de César Bernal; que plantea las siguientes etapas: Fase preparatoria que hace correlación a la etapa reflexiva y conformación cultural del investigador; en este sentido fue fundamental la contextualización teórica, conceptual y legal de la temática estudiada. En la Fase de diseño, se estableció el uso del enfoque cuantitativo de tipo descriptivo. Por su parte, la Fase de ejecu

In [8]:
# show with pretty format
data[0].page_content.split("\n\n") # select the correct splitter character

['https://doi.org/10.23850/2422068X.1182',
 'Un análisis de la estructura productiva del Alto Magdalena',
 'An analysis of the productive structure of Alto Magdalena',
 'dIana PatrIcIa GaLIndo áVILa1 GLorIa ISaBeL PrecIado trUJILLo2',
 'RESUMEN Este artículo da a conocer el desarrollo del proyecto de investigación que permitió la caracterización empresarial de la provincia del Alto Magdalena, cuyo objetivo es determinar los elementos esenciales que influyen en el crecimiento económico de la región.',
 'Dentro del desarrollo de la investigación se usó como referente metodológico el libro Metodología de la investigación de César Bernal; que plantea las siguientes etapas: Fase preparatoria que hace correlación a la etapa reflexiva y conformación cultural del investigador; en este sentido fue fundamental la contextualización teórica, conceptual y legal de la temática estudiada. En la Fase de diseño, se estableció el uso del enfoque cuantitativo de tipo descriptivo. Por su parte, la Fase de

In [23]:
# number of characters in raw document
print(f"N chars in raw doc: {len(data[0].page_content)}")

N chars in raw doc: 27907


### RAG Configuration

download embedding model and chat model (mistral)

In [ ]:
!ollama pull nomic-embed-text
!ollama pull mistral
#!ollama list --uncomment this line for validate all models were downloaded

Install dependencies for local vector database and text splitters

In [ ]:
%pip install --q chromadb
%pip install --q langchain-text-splitters

In [ ]:
# load langchain dependencies for use ollama Embeddings and Chromadb
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [ ]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100) # select the properly chunk size 
chunks = text_splitter.split_documents(data)

In [ ]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

### Retrieve

In [ ]:
# dependencies for chatting and chaining
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [ ]:
# LLM from Ollama
local_model = "mistral"
llm = ChatOllama(model=local_model)

In [ ]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate three
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [ ]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke(input(""))

In [ ]:
chain.invoke("Qué dice la teoría del lugar central?")

In [ ]:
# Delete all collections in the db
vector_db.delete_collection()